In [ ]:
from langgraph.graph import StateGraph , START , END
from typing import TypedDict 
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
import os
api_key = os.environ.get("PERPLEXITY_API_KEY")
llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.perplexity.ai",
    model="sonar-pro",
    # or another supported Perplexity model
)

In [13]:
from langgraph.checkpoint.memory import InMemorySaver

In [14]:
class JokeState(TypedDict):
    topic : str
    joke : str
    explaination : str  

In [15]:
def generate_joke(state:JokeState)->JokeState:
    prompt = f'Write a joke about {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke':response}

def generate_explaination(state:JokeState)->JokeState:
    prompt = f'Explain the joke "{state["joke"]}"'
    response = llm.invoke(prompt)

    return {'explaination':response.content}

In [16]:
#create graph
graph = StateGraph(JokeState)
#create nodes 
graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explaination", generate_explaination )
#create edges
graph.add_edge(START , "generate_joke")
graph.add_edge("generate_joke" , "generate_explaination")
graph.add_edge("generate_explaination" , END)

# add checkpointer object 
checkpointer = InMemorySaver()

#compile the graph
workflow = graph.compile(checkpointer=checkpointer)


In [ ]:
config1 = {"configurable":{"thread_id":"1"}}
workflow.invoke({'topic':'dogs'} , config=config1)

workflow.get_state(config1)

StateSnapshot(values={'topic': 'dogs', 'joke': 'Why did the dog sit in the shade? Because he didn’t want to be a hot dog[2][3][4].\n\nFor more dog laughs:\n- What do you call a dog magician? **A labracadabrador**[2].\n- Why are dogs such bad dancers? **They have two left feet**[2][3].\n- What kind of dog never throws anything away? **A hoarder collie**[3].\n\nThese playful puns capture the humor that dog lovers enjoy about their furry friends.', 'explaination': 'The joke "Why did the dog sit in the shade? Because he didn’t want to be a hot dog" is a play on words that uses the double meaning of "hot dog"[1][2]. The humor comes from the *pun*:\n\n- **"Hot dog"** can refer to a dog that is hot from being in the sun, but it is also the name for the popular food item (a sausage in a bun)[1].\n- The joke imagines a literal dog avoiding heat to not become a "hot dog" (as in an overheated canine), while also making a playful reference to the food (as if the dog doesn\'t want to become a sausa

In [22]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'dogs', 'joke': 'Why did the dog sit in the shade? Because he didn’t want to be a hot dog[2][3][4].\n\nFor more dog laughs:\n- What do you call a dog magician? **A labracadabrador**[2].\n- Why are dogs such bad dancers? **They have two left feet**[2][3].\n- What kind of dog never throws anything away? **A hoarder collie**[3].\n\nThese playful puns capture the humor that dog lovers enjoy about their furry friends.', 'explaination': 'The joke "Why did the dog sit in the shade? Because he didn’t want to be a hot dog" is a play on words that uses the double meaning of "hot dog"[1][2]. The humor comes from the *pun*:\n\n- **"Hot dog"** can refer to a dog that is hot from being in the sun, but it is also the name for the popular food item (a sausage in a bun)[1].\n- The joke imagines a literal dog avoiding heat to not become a "hot dog" (as in an overheated canine), while also making a playful reference to the food (as if the dog doesn\'t want to become a saus

In [ ]:
config2 = {"configurable":{"thread_id":"2"}}
workflow.invoke({'topic':'pasta'}, config=config2)


workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'What do you call a fake noodle?  \n**An impasta.**[1]\n\nFor extra context, pasta jokes often rely on wordplay and puns using pasta types or Italian food terminology, making them popular "dad jokes" as well as a light-hearted way to play with language[1][2]. If you’d like another, here’s a bonus:  \nDid you hear about the Italian chef who died?  \nHe **pasta** way[1][2].', 'explaination': 'The joke "What do you call a fake noodle?  \n**An impasta.**" is a pun combining the words **"imposter"** (someone who pretends to be someone else) and **"pasta"** (a type of noodle)[1][3][4]. The humor comes from the similar sounds of "imposter" and "impasta," making it sound like a word for a fake noodle—literally, an "im-pasta"[1][3][4].\n\nPasta jokes often rely on wordplay using pasta or Italian food terms, and are commonly found in the "dad joke" style—simple puns or one-liners that are inoffensive and easy to understand[1][4]. The bonus joke, "D

In [ ]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'What do you call a fake noodle?  \n**An impasta.**[1]\n\nFor extra context, pasta jokes often rely on wordplay and puns using pasta types or Italian food terminology, making them popular "dad jokes" as well as a light-hearted way to play with language[1][2]. If you’d like another, here’s a bonus:  \nDid you hear about the Italian chef who died?  \nHe **pasta** way[1][2].', 'explaination': 'The joke "What do you call a fake noodle?  \n**An impasta.**" is a pun combining the words **"imposter"** (someone who pretends to be someone else) and **"pasta"** (a type of noodle)[1][3][4]. The humor comes from the similar sounds of "imposter" and "impasta," making it sound like a word for a fake noodle—literally, an "im-pasta"[1][3][4].\n\nPasta jokes often rely on wordplay using pasta or Italian food terms, and are commonly found in the "dad joke" style—simple puns or one-liners that are inoffensive and easy to understand[1][4]. The bonus joke, "